In [1]:
import osmnx as ox, networkx as nx, matplotlib.cm as cm, pandas as pd, numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import functools
import community
import math


from scipy import sparse
from scipy.sparse import linalg
import time
from shapely.geometry import Point, LineString, Polygon, MultiPolygon, mapping, MultiLineString
from math import sqrt
import pandas as pd
from shapely.ops import cascaded_union
pd.set_option('precision', 10)
%matplotlib inline

In [3]:
import image_city_functions as ic

In [4]:
#initialise path, names, etc.

city_name = 'London'
folder_ouptut ='Outputs/'+city_name+'/'
epsg = 27700
crs = {'init': 'epsg:27700', 'no_defs': True}

# city_name = 'Boston'
# folder_output ='Outputs/'+city_name+'/'
# epsg = 26986
# crs = {'init': 'epsg:26986', 'no_defs': True}

# Downloading the data by place or Loading shapefile

In [ ]:
# 1 or 2 networks may be needed
# Option 1 ------------
# One graph is used (graph_A), when we consider a large area of study, e.g. an area of 10000 mt around the city centre (address). This area 
# is used or centrality measures as well as for districts extraction. Finally, edges, nodes and districts are extracted from 
# graph_A, by means of a spatial intersection, using the polygon of interest.

# Option 2 ------------
# 2 graphs: the one larger than the final area of interest, and a second one even more extended than the firs one, for a more
# complete district extraction. Finally, edges, nodes are extracted from graph_A, districts from graph_B by means of a spatial
# intersection, using the polygon of interest.


Option_1 = True
Option_2 = False

download = True
shapeFiles = False

In [6]:
# if the street network is too extended for the purposes, please specify the distance from the center of the area
# to scale down the area studied

if shapeFiles = True:
    
    if (Option_1 == True):
        directory_graphA = 'C:/Users/g_filo01/sciebo/GIS Data/Boston/Street_Network/boston_sn_26986_clipped_8000_noHW.shp'
        nodes_small, edges_small = ic.get_fromSHP(directory_graphA, epsg, crs, simplify = True, area = None)
        nodes_large, edges_large = nodes_large, edges_large   
        polygon_clip = 
    
    elif (Option_2 == True):
        directory_graphA = 'C:/Users/g_filo01/sciebo/GIS Data/Boston/Street_Network/boston_sn_26986_clipped_8000_noHW.shp'
        directory_graphB = 'C:/Users/g_filo01/sciebo/GIS Data/Boston/Street_Network/boston_sn_26986_clipped_8000_noHW.shp'
        nodes_small, edges_small = ic.get_fromSHP(directory_graphA, epsg, crs, simplify = True, area = None)
        # nodes_large, edges_large = ic.get_fromSHP(directory_graphB, epsg, crs, simplify = True 

In [3]:
if download = True:
    
    if (Option_1 == True):
    
        # choose between: (OSMplace, OSMpolygon, distance_from_address, shapefilPolygon)
        method_A = 'distance_from_address' 
#         place_A = 'C:/Users/g_filo01/sciebo/GIS Data/London/Congestion_area_polygon/Polygon.shp'
        place_A = 'Temple Place, London, UK'
        nodes_small, edges_small = ic.get_fromOSM(method_A, place_A, 'walk')
        nodes_large, edges_large = nodes_large, edges_large   
        
    
    elif (Option_2 == True):
    
        # choose between: (OSMplace, OSMpolygon shapefilPolygon)
        method_A = 'OSMpolygon' 
        place_A = 'C:/Users/g_filo01/sciebo/GIS Data/London/Congestion_area_polygon/Polygon.shp'


        method_B = 'shapefilePolygon' 
        place_B = 'Greater London (175342)'
        
        nodes_small, edges_small = ic.get_fromOSM(method_A, place_A, 'walk')
        nodes_large, edges_large = ic.get_fromOSM(method_B, place_B, 'walk')

SyntaxError: invalid syntax (<ipython-input-3-db8bc9b440cc>, line 1)

In [ ]:
if (city_name == 'Boston'):
    directory_small = 
    directory_large = 'C:/Users/g_filo01/sciebo/GIS Data/Boston/Street_Network/boston_sn_26986_clipped_8000_noHW.shp'

else:
    directory_small = 'C:/Users/g_filo01/sciebo/GIS Data/London/OS_Roads_Working/OS_roads_final.shp'
    directory_large = 'C:/Users/g_filo01/sciebo/GIS Data/Boston/Street_Network/boston_sn_26986_clipped_8000_noHW.shp'

# Loading a Street Network - gdf

In [ ]:
# nodes_small, edges_small = ic.get_fromSHP(directory_small, epsg, crs)
nodes_small, edges_small = ic.get_fromSHP(directory_small, epsg, crs, simplify = True, area = 1500)
# nodes_large, edges_large = ic.get_fromSHP(directory_large, epsg)

# Indexes computation - Nodes
## only area of analysis

In [ ]:
paths = gpd.read_file("Outputs/"+city_name+"/"+city_name+"_paths.shp")

In [ ]:
attributes_nodes = [] # besides nodeID
edges_costs = ['length']

NGs = ic.graph_fromGDF(nodes_small, edges_small, attributes_nodes, edges_costs)
# NGl = ic.graph_fromGDF(nodes_large, edges_large)

## Centrality measures

In [ ]:
Bc = nx.betweenness_centrality(NGs, k=None, weight = 'length', normalized=False)
# Sc = ic.straightness_centrality(NGs, weight = 'length', normalized=False)

In [ ]:
# POI = gpd.read_file('Outputs/POI.shp')
POI = (gpd.read_file(folder_ouptut+'/'+city_name+'_POI.shp')).to_crs(epsg=epsg)

index_geometry = nodes_small.columns.get_loc("geometry")+1
for row in nodes_small.itertuples():
    
    g = row[index_geometry] #geometry
    fil = g.buffer(50)
    
    sindex = POI.sindex
    possible_matches_index = list(sindex.intersection(fil.bounds))
    possible_matches = POI.iloc[possible_matches_index]
        
    weight = len(possible_matches)
    nodes_small.set_value(row[0], 'weight', weight)
    
for n in NGs.nodes():
    NGs.node[n]['weight'] = nodes_small['weight'].loc[n]

In [ ]:
# Reach Centrality
Rc400 = ic.reach_centrality(NGs, weight = 'length', radius = 400) 
Rc600 = ic.reach_centrality(NGs, weight = 'length', radius = 600)

In [ ]:
# Local Betweenness Centrality
Bc600 = ic.local_betweenness_centrality(NGs, weight = 'length', radius = 600) 
Bc400 = ic.local_betweenness_centrality(NGs, weight = 'length', radius = 800)

In [ ]:
nodes_df = ic.dict_to_df([Bc], ['Bc'])
nodes = pd.merge(nodes_small, nodes_df, left_on= "nodeID", right_index = True, how='left')

In [ ]:
col = [Bc, Sc, Rc400, Rc600, Bc400, Bc600]
names = ['Bc', 'Sc', 'Rc400', 'Rc600', 'Bc400', 'Bc600']

nodes_df = ic.dict_to_df(col, names)
nodes = pd.merge(nodes_small, nodes_df, left_on= "nodeID", right_index = True, how='left')

for i in names: ic.scaling_columnDF(nodes, i)
nodes['height'] = 2 # for 3d visibility analysis
nodes.head()

In [ ]:
nodes.crs = crs
nodes.to_file(folder_ouptut+city_name+'_experiment_nodes.shp', driver='ESRI Shapefile')
# nodes_small.to_file(folder_ouptut+city_name+'_nodes.shp', driver='ESRI Shapefile')

# Paths

In [ ]:
Eb = nx.edge_betweenness_centrality(NGs, weight= 'rad', normalized=False)

In [ ]:
edge_id = {}

for i, g in NGs.edges(): edge_id[(i,g)] = NGs[i][g]['streetID']

edges_df = ic.dict_to_df([Eb, edge_id], ["Eb", "streetID"])
edges_df.streetID = edges_df.streetID.astype(int)

In [ ]:
paths_tmp = pd.merge(edges_small, edges_df, left_on = 'streetID', right_on = 'streetID', how='left')

# Dual Analysis

In [ ]:
nodesDual_small, edgesDual_small = ic.dual_gdf(nodes_small, edges_small, crs)
# nodesDual_large, edgesDual_large = ic.dual_gdf(nodes_large, edges_large, crs)

In [ ]:
nodesDual_small = nodesDual_small.drop('intersecting',axis=1)
nodesDual_small.crs = crs
nodesDual_small.to_file(folder_ouptut+city_name+'_experiment_nodesDual.shp', driver='ESRI Shapefile')

In [ ]:
ic.scaling_columnDF(edgesDual_small, 'rad') # for combining with landmarkness
edgesDual_small.crs = crs
edgesDual_small.to_file(folder_ouptut+city_name+'_experiment_edgesDual.shp', driver='ESRI Shapefile')

## Creating the dual graph

In [ ]:
edges_costs = ['rad', 'length', 'deg']
DG_small = ic.get_dual_graph(nodesDual_small, edgesDual_small, edges_costs)
# DG_large =  ic.get_dual_graph(nodesDual_large, edgesDual_large)

## Angular Betweenness - Paths

In [ ]:
Ab = nx.betweenness_centrality(DG_small, weight ='angle', normalized=False)
Ab_dict = ic.dual_id_dict(Ab, DG_small, 'streetID')
Ab_df = ic.dict_to_df([Ab_dict], ['Ab'])

In [ ]:
paths = pd.merge(paths_tmp, Ab_df, left_on = "streetID", right_index=True, how='left')

col = ['Eb', 'Ab']
for i in col: ic.scaling_columnDF(paths, i)

## Partitions - Districts

In [ ]:
weights = ['length', 'rad', 'False']
subdvisions = []

for i in weights:
    partition = community.best_partition(DG_large, weight=i)
    dct = ic.dual_id_dict(partition, DG_large, 'streetID')
    subdvisions.append(dct)

In [ ]:
partitions_df = ic.dict_to_df(subdvisions, ["p_len", "p_rad", 'p_no'])

In [ ]:
districts = pd.merge(edges_large, partitions_df, left_on= "streetID", right_index = True, how= 'left')
districts.to_file(folder_ouptut+city_name+'_districts.shp', driver='ESRI Shapefile')

# Natural Roads

In [ ]:
paths['natural_id'] = "NA"
natural_id = 0 

for row in paths.itertuples():

    if (row[-1] != 'NA'): continue # if already assigned to a natural road
   
    ic.natural_roads(row[0], natural_id, "fr", paths, nodes) # assuming streetID = index
    ic.natural_roads(row[0], natural_id, "to", paths, nodes) 
    natural_id = natural_id+1

In [ ]:
ic.scaling_columnDF(paths, 'length')
paths.crs = crs
paths.to_file(folder_ouptut+city_name+'_experiment_paths.shp', driver='ESRI Shapefile')